In [0]:
spark.conf.set("spark.sql.shuffle.partitions", sc.defaultParallelism*2)

In [0]:
%pip install azure-storage-file-datalake

Python interpreter will be restarted.
Collecting azure-storage-file-datalake
 Downloading azure_storage_file_datalake-12.6.0-py3-none-any.whl (211 kB)
Collecting azure-storage-blob<13.0.0,>=12.10.0
 Using cached azure_storage_blob-12.11.0-py3-none-any.whl (346 kB)
Collecting azure-core<2.0.0,>=1.15.0
 Using cached azure_core-1.23.1-py3-none-any.whl (178 kB)
Collecting msrest>=0.6.21
 Using cached msrest-0.6.21-py2.py3-none-any.whl (85 kB)
Requirement already satisfied: requests>=2.18.4 in /databricks/python3/lib/python3.8/site-packages (from azure-core<2.0.0,>=1.15.0->azure-storage-file-datalake) (2.25.1)
Collecting typing-extensions>=4.0.1
 Using cached typing_extensions-4.1.1-py3-none-any.whl (26 kB)
Requirement already satisfied: six>=1.11.0 in /databricks/python3/lib/python3.8/site-packages (from azure-core<2.0.0,>=1.15.0->azure-storage-file-datalake) (1.15.0)
Collecting cryptography>=2.1.4
 Using cached cryptography-36.0.2-cp36-abi3-manylinux_2_24_x86_64.whl (3.6 MB)
Requirement already satisfied: cffi>=1.12 in /databricks/python3/lib/python3.8/site-packages (from cryptography>=2.1.4->azure-storage-blob<13.0.0,>=12.10.0->azure-storage-file-datalake) (1.14.5)
Requirement already satisfied: pycparser in /databricks/python3/lib/python3.8/site-packages (from cffi>=1.12->cryptography>=2.1.4->azure-storage-blob<13.0.0,>=12.10.0->azure-storage-file-datalake) (2.20)
Collecting isodate>=0.6.0
 Using cached isodate-0.6.1-py2.py3-none-any.whl (41 kB)
Requirement already satisfied: certifi>=2017.4.17 in /databricks/python3/lib/python3.8/site-packages (from msrest>=0.6.21->azure-storage-file-datalake) (2020.12.5)
Collecting requests-oauthlib>=0.5.0
 Using cached requests_oauthlib-1.3.1-py2.py3-none-any.whl (23 kB)
Requirement already satisfied: chardet<5,>=3.0.2 in /databricks/python3/lib/python3.8/site-packages (from requests>=2.18.4->azure-core<2.0.0,>=1.15.0->azure-storage-file-datalake) (4.0.0)
Requirement already satisfied: urllib3<1.27,>=1.21.1 in /databricks/python3/lib/python3.8/site-packages (from requests>=2.18.4->azure-core<2.0.0,>=1.15.0->azure-storage-file-datalake) (1.25.11)
Requirement already satisfied: idna<3,>=2.5 in /databricks/python3/lib/python3.8/site-packages (from requests>=2.18.4->azure-core<2.0.0,>=1.15.0->azure-storage-file-datalake) (2.10)
Collecting oauthlib>=3.0.0
 Using cached oauthlib-3.2.0-py3-none-any.whl (151 kB)
Installing collected packages: oauthlib, typing-extensions, requests-oauthlib, isodate, msrest, cryptography, azure-core, azure-storage-blob, azure-storage-file-datalake
Successfully installed azure-core-1.23.1 azure-storage-blob-12.11.0 azure-storage-file-datalake-12.6.0 cryptography-36.0.2 isodate-0.6.1 msrest-0.6.21 oauthlib-3.2.0 requests-oauthlib-1.3.1 typing-extensions-4.1.1
WARNING: You are using pip version 21.0.1; however, version 22.0.4 is available.
You should consider upgrading via the '/local_disk0/.ephemeral_nfs/envs/pythonEnv-af7d1f4f-a76b-4e45-8495-516d8f506a42/bin/python -m pip install --upgrade pip' command.
Python interpreter will be restarted.

In [0]:
import datetime 
from azure.storage.filedatalake import DataLakeServiceClient

account_key = dbutils.secrets.get(scope ="am-da-kv-general" ,key="General-Datalake-Account-Key")
storage_account_name = dbutils.secrets.get(scope ="am-da-kv-general" ,key="am-da-s-general-adl-name")

try:  
    global service_client
        
    service_client = DataLakeServiceClient(account_url="{}://{}.dfs.core.windows.net".format(
        "https", storage_account_name), credential=account_key)
    
except Exception as e:
    print(e)

file_system_client = service_client.get_file_system_client(file_system="raw")
#paths = file_system_client.get_paths(path="batch/sales/bigmachine/commerce_v1/quote_header")

# manual date entry 
# from_date = datetime.datetime.strptime('2020-09-10', "%Y-%m-%d")

def getLatestFiles(start_from , directory_path):
  paths = file_system_client.get_paths(path=directory_path)
  output = []
  for path in paths:
    fileLastModifiedDatetime = datetime.datetime.strptime(str(path.last_modified),  "%Y-%m-%d %H:%M:%S")
    if fileLastModifiedDatetime >= start_from and (path.name.endswith(".csv") or  path.name.endswith(".json")):
        output.append("/mnt/datalake_raw/"+path.name)
  return output 

#start_date= datetime.datetime.combine(datetime.datetime.today(), datetime.time(0, 0, 0, 0))
start_date = datetime.datetime.today() -  datetime.timedelta(hours = 72)
# subjects = ['account','contact','service_contract','user','lead','opportunity','opportunity_product','ib_unit','deleted_items','activity','existing_quote']
# files = dict()
# for subject in subjects:
#   files[subject] = getLatestFiles(start_from = start_date  , directory_path = "batch/sales/oraclecrm/{subject}/delta".format(subject=subject))

quote_header_v1_list = getLatestFiles(start_from = start_date  , directory_path = "batch/sales/bigmachine/commerce_v1/quote_header")
quote_header_v2_list = getLatestFiles(start_from = start_date  , directory_path = "batch/sales/bigmachine/commerce_v2/quote_header")
quote_line_v1_list = getLatestFiles(start_from = start_date  , directory_path = "batch/sales/bigmachine/commerce_v1/quote_line")
quote_line_v2_list = getLatestFiles(start_from = start_date  , directory_path = "batch/sales/bigmachine/commerce_v2/quote_line")

if len(quote_header_v2_list) + len(quote_header_v1_list) == 0:
  dbutils.notebook.exit("No new deltas have been captured")

In [0]:
"""
Quote Header
"""


from pyspark.sql.types import StringType , TimestampType , DoubleType , StructType , StructField , LongType
from pyspark.sql.functions import col  , year , to_timestamp , to_date , row_number , lit
from pyspark.sql import Window
from delta.tables import *  

quoteSchema = StructType([
  
                                          StructField("crmBranch_quote",StringType(), True),
                          StructField("crmRegion_quote",StringType(), True),
                          StructField("lineOfBusiness_quote",StringType(), True),
                          StructField("sublineOfBusiness_quote",StringType(), True),
                          StructField("crmOpportunityId_quote",StringType(), True),
                          StructField("quoteNumber_quote",StringType(), True),
                          StructField("status_quote",StringType(), True),
                          StructField("_reconfiguration_date",TimestampType(), True),
                          StructField("createdDate_quote",TimestampType(), True),
                          StructField("orderedDate_quote",TimestampType(), True),
                          StructField("submittedDate_quote",TimestampType(), True),
                          StructField("transactionID_quote",StringType(), True),
                          StructField("branchLaborEfficiency_quote",StringType(), True),
                          StructField("_date_modified",TimestampType(), True) ,
                          StructField("oracleBranchID_quote",StringType(),True) ,
                          StructField("_customer_id",StringType(),True) ,
                          StructField("CRMSalesStage_quote",StringType(),True) ,
                          StructField("crmSalesRepId_quote",StringType(),True) ,
                          StructField("crmSalesRep_quote",StringType(),True) ,
                          StructField("proposedDate_quote",TimestampType(),True),
                          StructField("jobStatus_quote",StructType([StructField("value", StringType() , True)]),True), # added on 3/25/2021 sprint #6 
                          #StructField("maxLink_quote",DoubleType(),True) , # added on 3/25/2021 sprint #6 
                          StructField("multimediaMonitoring_quote",DoubleType(),True) , # added on 3/25/2021 sprint #6 
                          StructField("_currency_pref",StructType([StructField("currencyCode",StringType(),True),StructField("id",LongType(),True)])),                          
                          StructField("aGILEMobileCoverage_quote",StringType(), True),	
                          StructField("beaconCoverage_quote",StringType(), True),	
                          StructField("NoOfBeacons_quote",StringType(), True),	
                          StructField("beaconCoverageAmount_quote" ,DoubleType() , True),
                          StructField("aGILEMobileCoverageAmount_quote",StringType(), True),	
                          StructField("NoOfAGILEMobileUsers_quote",StringType(), True),	
                          StructField("NoOfAGILEMobileUnits_quote",StringType(), True),	
                          StructField("NumOfMAXEligibleUnits_quote",StringType(), True),	
                          StructField("removeMAXServices_quote",StringType(), True),	
                          StructField("NumberOfMaxPro_quote",StringType(), True),	
                          StructField("NumberOfMaxPlus_quote",StringType(), True),	
                          StructField("NumberOfMaxPremium_quote",StringType(), True),	
                         StructField("CurrencyOfMaxSelections_quote",StructType([StructField("value", DoubleType() , True)]), True),	
                          StructField("NoOfBuildingsForMAXActivation_quote",StringType(), True),	
                          StructField("removeMAX30DayCancellationClause_quote",StringType(), True),	
                          StructField("mAXCOVID19",StringType(), True),	
                          StructField("mAXActivationFee_quote" ,StructType([StructField("value", DoubleType() , True) , StructField("currency" , StringType() , True)])),
                          StructField("maxLink_quote",StringType(), True),	
                          StructField("maxLinkAmount_quote" ,StructType([StructField("value", DoubleType() , True) , StructField("currency" , StringType() , True)])),
                          StructField("communicationsMonitoring_quote",StructType([StructField("value", DoubleType() , True)]), True),
                          StructField("crmPriceEscalationCapType_quote",StructType([StructField("value", StringType() , True)]),True),
                          StructField("crmPriceEscalationCapValue_quote",DoubleType(), True),
                          StructField("estimateContainsConfig30Models_quote",StringType() , True),
                          StructField("creator_quote",StringType() , True),
                          StructField("preparedByName_quote",StringType() , True),
                          StructField("preparedByEmail_quote",StringType() , True),
                          StructField("totalHighCabSubsidy_quote",DoubleType(),True) # added on 3/22/202 sprint ####
                           ] 
                           )


emptyDF = (sqlContext.createDataFrame(sc.emptyRDD(), quoteSchema)
           .withColumn("SourceSystem",lit("Commerce"))
           .withColumn('isDeleted',lit(0))
           .withColumn("currency_pref",col("_currency_pref.currencyCode"))                        
          .select("*",col("jobStatus_quote.value").alias("jobStatus")
         
          ,col("mAXActivationFee_quote.value").alias("mAXActivationFee")
          ,col("maxLinkAmount_quote.value").alias("maxLinkAmount")
                  ,col("CurrencyOfMaxSelections_quote.value").alias("CurrencyOfMaxSelections")
                  ,col("crmPriceEscalationCapType_quote.value").alias("crmPriceEscalationCapType")
                  ,col("communicationsMonitoring_quote.value").alias("communicationsMonitoring")

          )
 .drop("_currency_pref","jobStatus_quote","mAXActivationFee_quote","maxLinkAmount_quote","crmPriceEscalationCapType_quote","communicationsMonitoring_quote","CurrencyOfMaxSelections_quote")
          )

if quote_header_v1_list:
  Quote_Header_v1 = (
                        spark.read
                          .schema(quoteSchema)
                          .option("multiline","true")
                          .option("timestampFormat","MM/dd/yyyy HH:mm:SS a")
                          #.json("/mnt/datalake_raw/batch/sales/bigmachine/commerce_v1/quote_header/*/*.json")
                          .json(quote_header_v1_list)
                          .withColumn("SourceSystem",lit("Commerce_v1"))
                          .withColumn('isDeleted',lit(0))
                          .withColumn("currency_pref",col("_currency_pref.currencyCode"))
                          .select("*",col("jobStatus_quote.value").alias("jobStatus")
                            
                                  ,col("mAXActivationFee_quote.value").alias("mAXActivationFee")
                                  ,col("maxLinkAmount_quote.value").alias("maxLinkAmount")
                                  ,col("CurrencyOfMaxSelections_quote.value").alias("CurrencyOfMaxSelections")
                                  ,col("crmPriceEscalationCapType_quote.value").alias("crmPriceEscalationCapType")
                                  ,col("communicationsMonitoring_quote.value").alias("communicationsMonitoring")
   
                                 )
.drop("_currency_pref","jobStatus_quote","mAXActivationFee_quote","maxLinkAmount_quote","crmPriceEscalationCapType_quote","communicationsMonitoring_quote","CurrencyOfMaxSelections_quote")
                     )
else:
  Quote_Header_v1 = emptyDF

if quote_header_v2_list:  
  Quote_Header_v2 = (
                        spark.read
                          .schema(quoteSchema)
                          .option("multiline","true")
                          .option("timestampFormat","MM/dd/yyyy HH:mm:SS a")
                          #.json("/mnt/datalake_raw/batch/sales/bigmachine/commerce_v2/quote_header/*/*.json")
                          .json(quote_header_v2_list)
                          .withColumn("SourceSystem",lit("Commerce_v2"))
                          .withColumn('isDeleted',lit(0))
                          .withColumn("currency_pref",col("_currency_pref.currencyCode"))
                          .select("*",col("jobStatus_quote.value").alias("jobStatus")
                                
                                  ,col("mAXActivationFee_quote.value").alias("mAXActivationFee")
                                  ,col("maxLinkAmount_quote.value").alias("maxLinkAmount")
                                  ,col("CurrencyOfMaxSelections_quote.value").alias("CurrencyOfMaxSelections")
                                  ,col("crmPriceEscalationCapType_quote.value").alias("crmPriceEscalationCapType")
                                  ,col("communicationsMonitoring_quote.value").alias("communicationsMonitoring")
                                
                                 )
.drop("_currency_pref","jobStatus_quote","mAXActivationFee_quote","maxLinkAmount_quote","crmPriceEscalationCapType_quote","communicationsMonitoring_quote","CurrencyOfMaxSelections_quote")
                       )
else:
  Quote_Header_v2 = emptyDF
  

Quote_Header = Quote_Header_v1.unionAll(Quote_Header_v2)
        
window = Window.partitionBy("SourceSystem","transactionID_quote").orderBy(Quote_Header["_date_modified"].desc())

quote_header_update = (Quote_Header.dropDuplicates().withColumn("RowNumber" , row_number().over(window))
 .filter("RowNumber == 1")
 .drop("RowNumber")
 .withColumn("CreatedYear",year(col("createdDate_quote").cast("date"))))

# Merge User: Update changed quotes and insert newly created accounts 

quote_header_g = DeltaTable.forPath(spark, "/mnt/datalake_premium/cpq/quote_header") 

quote_header_g.alias("t").merge(
      quote_header_update.alias("s"),
      "t.transactionID_quote = s.transactionID_quote and t.SourceSystem = s.SourceSystem") \
    .whenNotMatchedInsertAll() \
    .whenMatchedUpdateAll() \
    .execute()

In [0]:
"""
Quote Line
"""


from pyspark.sql.types import StringType , TimestampType , DoubleType , StructType , StructField
from pyspark.sql.functions import col  , year , to_timestamp , to_date , row_number , lit
from pyspark.sql import Window
from delta.tables import *  

quoteLineSchema = StructType([
                     StructField("_bs_id", StringType() , False),
                    StructField("_id", StringType() , False),
                  StructField("_sequence_number", StringType() , False),
                    StructField("lineDocNum_line", StringType() , False),
                  StructField("buildingName_line", StringType() , True),
                  StructField("lineType_line", StructType([StructField("value", StringType() , True)])), # added on 3/24/2021 - sprint #6 
                  StructField("crmIntegrationProductName_line", StringType() , True),
                  StructField("crmNumberOfFrontOpenings_line", StringType() , True),
                  StructField("crmNumberOfRearOpenings_line", StringType() , True),
                  StructField("crmNumberOfStops_line", StringType() , True),
                  StructField("crmSpeed_line", StringType() , True),
                  StructField("crmCapacity_line", StringType() , True), # added on 4/21/2021 - Sprint #8		
                  StructField("factoryJobNumber_line", StringType() , True),
                  StructField("itemDescription_line", StringType() , True),
                   StructField("itemType_line", StringType() , True),
                   StructField("marginAmount_line" ,StructType([StructField("value", DoubleType() , True) , StructField("currency" , StringType() , True)])),
                  StructField("marginPercentage_line" , StringType() , True),
                   StructField("markedUpPrice_line" ,StructType([StructField("value", DoubleType() , True) , StructField("currency" , StringType() , True)])),
                  StructField("oemSerialNumber_line", StringType() , True),
                  StructField("oracleSerialNumber_line", StringType() , True),
                  StructField("ozProduct_line", StringType() , True),
                   StructField("proposalPricePerUnit_line" ,StructType([StructField("value", DoubleType() , True) , StructField("currency" , StringType() , True)])),
                   StructField("proposalPrice_line" ,StructType([StructField("value", DoubleType() , True) , StructField("currency" , StringType() , True)])),
                   StructField("totalCost_line" ,StructType([StructField("value", DoubleType() , True) , StructField("currency" , StringType() , True)])),
                  StructField("totalLaborHours_line" , StringType() , True),
                  StructField("totalTeamLaborHours_line" , StringType() , True),
                  StructField("unitLaborCost_line" ,StructType([StructField("value", DoubleType() , True) , StructField("currency" , StringType() , True)])),
                  StructField("unitMaterialCost_line" ,StructType([StructField("value", DoubleType() , True) , StructField("currency" , StringType() , True)])),
                   StructField("unitTotalLaborHours_line" , StringType() , True),
                  StructField("unitTravel_line", StringType() , True),
                  StructField("voltage_line", StringType() , True),
                  StructField("wBSTemplate_line", StringType() , True),
                  StructField("modelLevelConstructionPrice_line" ,StructType([StructField("value", DoubleType() , True) , StructField("currency" , StringType() , True)])),
                  StructField("extraExpenses_line" ,StructType([StructField("value", DoubleType() , True) , StructField("currency" , StringType() , True)])),
                   StructField("grossMarginWithoutOverhead_line" , StringType() , True),
                  StructField("_part_id", StringType() , True),
                  StructField("_model_id", StringType() , True),
                  StructField("productCode_line", StringType() , True),
                  StructField("productDescriptionUnitDescCRM_line", StringType() , True),
                  StructField("productLine_line", StringType() , True),
                  StructField("productType_line", StringType() , True),
                  StructField("_part_number", StringType() , True),
                  StructField("_part_supplier_company_id", StringType() , True),
                  StructField("_part_supplier_company_name", StringType() , True),
                  StructField("_model_name", StringType() , True),
                  StructField("_model_product_line_id", StringType() , True),
                  StructField("_model_product_line_name", StringType() , True),
                  StructField("_model_segment_id", StringType() , True),
                  StructField("_model_supplier_company_id", StringType() , True),
                  StructField("_model_supplier_company_name", StringType() , True),
                  StructField("laborEfficiencyPercentage_line", StringType() , True),
                  StructField("_date_modified",TimestampType(), True) ,
                  StructField("_date_added",TimestampType(), True)  ,
                  StructField("productConfiguration_line", StringType() , True),
  StructField("freightShippingCharges_line" ,StructType([StructField("value", DoubleType() , True) , StructField("currency" , StringType() , True)])),
  StructField("transportCosts_line" ,StructType([StructField("value", DoubleType() , True) , StructField("currency" , StringType() , True)])),
  StructField("inspectionPermitFees_line" ,StructType([StructField("value", DoubleType() , True) , StructField("currency" , StringType() , True)])),
  StructField("miscellaneousExpenses_line" ,StructType([StructField("value", DoubleType() , True) , StructField("currency" , StringType() , True)])),
  StructField("useTax_line" ,StructType([StructField("value", DoubleType() , True) , StructField("currency" , StringType() , True)])),
  StructField("factorTax_line" ,StructType([StructField("value", DoubleType() , True) , StructField("currency" , StringType() , True)])),
  StructField("unitDesignation_line", StringType() , True) ,
  StructField("numOfUnitsOnTheEstimate_line", StringType() , True) ,
  StructField("totalNumberOfCarsInBank_line", StringType() , True) ,

                           ] 
                           )
emptyDF = (sqlContext.createDataFrame(sc.emptyRDD(), quoteLineSchema).withColumn("SourceSystem",lit("Commerce")).select("_bs_id"
                        ,"_id"
                        ,"_sequence_number"
                        ,"lineDocNum_line"
                        ,"buildingName_line"
                        ,col("lineType_line.value").alias("lineType_line")  # added on 3/24/2021 - sprint #6 
                        ,"crmIntegrationProductName_line"
                        ,"crmNumberOfFrontOpenings_line"
                        ,"crmNumberOfRearOpenings_line"
                        ,"crmNumberOfStops_line"
                        ,"crmSpeed_line"
                        ,"factoryJobNumber_line"
                        ,"itemDescription_line"
                        ,"itemType_line"
                        ,col("marginAmount_line.value").alias("marginAmount_line")
                        ,"marginPercentage_line"
                        ,col("markedUpPrice_line.value").alias("markedUpPrice_line")
                        ,"oemSerialNumber_line"
                        ,"oracleSerialNumber_line"
                        ,"ozProduct_line"
                        ,col("proposalPricePerUnit_line.value").alias("proposalPricePerUnit_line")
                        ,col("proposalPrice_line.value").alias("proposalPrice_line")
                        ,col("totalCost_line.value").alias("totalCost_line")
                        ,"totalCost_line.currency"
                        ,"totalLaborHours_line"
                        ,"totalTeamLaborHours_line"
                        ,col("unitLaborCost_line.value").alias("unitLaborCost_line")
                        ,col("unitMaterialCost_line.value").alias("unitMaterialCost_line")
                        ,"unitTotalLaborHours_line"
                        ,"unitTravel_line"
                        ,"voltage_line"
                        ,"wBSTemplate_line"
                        ,col("modelLevelConstructionPrice_line.value").alias("modelLevelConstructionPrice_line")
                        ,col("extraExpenses_line.value").alias("extraExpenses_line")                                                                                         
                        ,"grossMarginWithoutOverhead_line"
                        ,"_part_id"
                        ,"_model_id"
                        ,"productCode_line"
                        ,"productDescriptionUnitDescCRM_line"
                        ,"productLine_line"
                        ,"productType_line"
                        ,"_part_number"
                        ,"_part_supplier_company_id"
                        ,"_part_supplier_company_name"
                        ,"_model_name"
                        ,"_model_product_line_id"
                        ,"_model_product_line_name"
                        ,"_model_segment_id"
                        ,"_model_supplier_company_id"
                        ,"_model_supplier_company_name"
                        ,"laborEfficiencyPercentage_line"
                        ,"_date_modified"
                        ,"_date_added"
                        ,"SourceSystem"
                        ,"productConfiguration_line"                                                                                                                                                      
                                                                                                                        
                               ,col("freightShippingCharges_line.value").alias("freightShippingCharges_line")
                               ,col("transportCosts_line.value").alias("transportCosts_line")
                               ,col("inspectionPermitFees_line.value").alias("inspectionPermitFees_line")
                               ,col("miscellaneousExpenses_line.value").alias("miscellaneousExpenses_line")
                               ,col("useTax_line.value").alias("useTax_line")
                              ,col("factorTax_line.value").alias("factorTax_line")
                              ,"unitDesignation_line"
                              ,"numOfUnitsOnTheEstimate_line"
                              ,"totalNumberOfCarsInBank_line"
                              ,"crmCapacity_line" # added on 4/21/2021 - Sprint #8	
                              , lit(0).alias("isDeleted")
 
              ))
if quote_line_v1_list:
  Quote_Line_v1 = (
                        spark.read
                           .schema(quoteLineSchema)
                          .option("multiline","true")
                          .option("timestampFormat","MM/dd/yyyy HH:mm:SS a")
                          #.json("/mnt/datalake_raw/batch/sales/bigmachine/commerce_v1/quote_line/*/*.json")
                          .json(quote_line_v1_list)
                          .withColumn("SourceSystem",lit("Commerce_v1"))
                          .select(                       
                          "_bs_id"
                          ,"_id"
                          ,"_sequence_number"
                          ,"lineDocNum_line"
                          ,"buildingName_line"
                          ,col("lineType_line.value").alias("lineType_line")  # added on 3/24/2021 - sprint #6 
                          ,"crmIntegrationProductName_line"
                          ,"crmNumberOfFrontOpenings_line"
                          ,"crmNumberOfRearOpenings_line"
                          ,"crmNumberOfStops_line"
                          ,"crmSpeed_line"
                          ,"factoryJobNumber_line"
                          ,"itemDescription_line"
                          ,"itemType_line"
                          ,col("marginAmount_line.value").alias("marginAmount_line")
                          ,"marginPercentage_line"
                          ,col("markedUpPrice_line.value").alias("markedUpPrice_line")
                          ,"oemSerialNumber_line"
                          ,"oracleSerialNumber_line"
                          ,"ozProduct_line"
                          ,col("proposalPricePerUnit_line.value").alias("proposalPricePerUnit_line")
                          ,col("proposalPrice_line.value").alias("proposalPrice_line")
                          ,col("totalCost_line.value").alias("totalCost_line")
                          ,"totalCost_line.currency"
                          ,"totalLaborHours_line"
                          ,"totalTeamLaborHours_line"
                          ,col("unitLaborCost_line.value").alias("unitLaborCost_line")
                          ,col("unitMaterialCost_line.value").alias("unitMaterialCost_line")
                          ,"unitTotalLaborHours_line"
                          ,"unitTravel_line"
                          ,"voltage_line"
                          ,"wBSTemplate_line"
                          ,col("modelLevelConstructionPrice_line.value").alias("modelLevelConstructionPrice_line")
                           ,col("extraExpenses_line.value").alias("extraExpenses_line")   
                          ,"grossMarginWithoutOverhead_line"
                          ,"_part_id"
                          ,"_model_id"
                          ,"productCode_line"
                          ,"productDescriptionUnitDescCRM_line"
                          ,"productLine_line"
                          ,"productType_line"
                          ,"_part_number"
                          ,"_part_supplier_company_id"
                          ,"_part_supplier_company_name"
                          ,"_model_name"
                          ,"_model_product_line_id"
                          ,"_model_product_line_name"
                          ,"_model_segment_id"
                          ,"_model_supplier_company_id"
                          ,"_model_supplier_company_name"
                          ,"laborEfficiencyPercentage_line"
                          ,"_date_modified"
                          ,"_date_added"
                          ,"SourceSystem"
                          ,"productConfiguration_line"
                                ,col("freightShippingCharges_line.value").alias("freightShippingCharges_line")
                               ,col("transportCosts_line.value").alias("transportCosts_line")
                               ,col("inspectionPermitFees_line.value").alias("inspectionPermitFees_line")
                               ,col("miscellaneousExpenses_line.value").alias("miscellaneousExpenses_line")
                               ,col("useTax_line.value").alias("useTax_line")
                              ,col("factorTax_line.value").alias("factorTax_line")
                              ,"unitDesignation_line"
                              ,"numOfUnitsOnTheEstimate_line"
                              ,"totalNumberOfCarsInBank_line"
                              ,"crmCapacity_line" # added on 4/21/2021 - Sprint #8	
                              , lit(0).alias("isDeleted")

                          )
                     )
else: 
  Quote_Line_v1 = emptyDF

if quote_line_v2_list:
  Quote_Line_v2 = (
                        spark.read
                           .schema(quoteLineSchema)
                          .option("multiline","true")
                          .option("timestampFormat","MM/dd/yyyy HH:mm:SS a")
                          #.json("/mnt/datalake_raw/batch/sales/bigmachine/commerce_v2/quote_line/*/*.json")
                          .json(quote_line_v2_list)
                          .withColumn("SourceSystem",lit("Commerce_v2"))
                          .select(

                          "_bs_id"
                          ,"_id"
                          ,"_sequence_number"
                          ,"lineDocNum_line"
                          ,"buildingName_line"
                          ,col("lineType_line.value").alias("lineType_line")  # added on 3/24/2021 - sprint #6 
                          ,"crmIntegrationProductName_line"
                          ,"crmNumberOfFrontOpenings_line"
                          ,"crmNumberOfRearOpenings_line"
                          ,"crmNumberOfStops_line"
                          ,"crmSpeed_line"
                          ,"factoryJobNumber_line"
                          ,"itemDescription_line"
                          ,"itemType_line"
                          ,col("marginAmount_line.value").alias("marginAmount_line")
                          ,"marginPercentage_line"
                          ,col("markedUpPrice_line.value").alias("markedUpPrice_line")
                          ,"oemSerialNumber_line"
                          ,"oracleSerialNumber_line"
                          ,"ozProduct_line"
                          ,col("proposalPricePerUnit_line.value").alias("proposalPricePerUnit_line")
                          ,col("proposalPrice_line.value").alias("proposalPrice_line")
                          ,col("totalCost_line.value").alias("totalCost_line")
                          ,"totalCost_line.currency"
                          ,"totalLaborHours_line"
                          ,"totalTeamLaborHours_line"
                          ,col("unitLaborCost_line.value").alias("unitLaborCost_line")
                          ,col("unitMaterialCost_line.value").alias("unitMaterialCost_line")
                          ,"unitTotalLaborHours_line"
                          ,"unitTravel_line"
                          ,"voltage_line"
                          ,"wBSTemplate_line"
                          ,col("modelLevelConstructionPrice_line.value").alias("modelLevelConstructionPrice_line")
                          ,col("extraExpenses_line.value").alias("extraExpenses_line")   
                          ,"grossMarginWithoutOverhead_line"
                          ,"_part_id"
                          ,"_model_id"
                          ,"productCode_line"
                          ,"productDescriptionUnitDescCRM_line"
                          ,"productLine_line"
                          ,"productType_line"
                          ,"_part_number"
                          ,"_part_supplier_company_id"
                          ,"_part_supplier_company_name"
                          ,"_model_name"
                          ,"_model_product_line_id"
                          ,"_model_product_line_name"
                          ,"_model_segment_id"
                          ,"_model_supplier_company_id"
                          ,"_model_supplier_company_name"
                          ,"laborEfficiencyPercentage_line"
                          ,"_date_modified"
                          ,"_date_added"
                          ,"SourceSystem"
                          ,"productConfiguration_line"
                               ,col("freightShippingCharges_line.value").alias("freightShippingCharges_line")
                               ,col("transportCosts_line.value").alias("transportCosts_line")
                               ,col("inspectionPermitFees_line.value").alias("inspectionPermitFees_line")
                               ,col("miscellaneousExpenses_line.value").alias("miscellaneousExpenses_line")
                               ,col("useTax_line.value").alias("useTax_line")
                              ,col("factorTax_line.value").alias("factorTax_line")
                              ,"unitDesignation_line"
                              ,"numOfUnitsOnTheEstimate_line"
                              ,"totalNumberOfCarsInBank_line"
                              ,"crmCapacity_line" # added on 4/21/2021 - Sprint #8
                            , lit(0).alias("isDeleted")

                          )

                     )
else:
  Quote_Line_v2 = emptyDF
  
Quote_Line = Quote_Line_v1.unionAll(Quote_Line_v2)
        
window = Window.partitionBy("SourceSystem","_id").orderBy(Quote_Line["_date_modified"].desc())


quote_line_update = (Quote_Line.dropDuplicates().withColumn("RowNumber" , row_number().over(window))
 .filter("RowNumber == 1")
 .drop("RowNumber")
 .withColumn("CreatedYear",year(col("_date_added").cast("date")))
)


# Merge quote line : Update changed quote lines and insert newly created accounts 

quote_line_g = DeltaTable.forPath(spark, "/mnt/datalake_premium/cpq/quote_line") 

quote_line_g.alias("t").merge(
      quote_line_update.alias("s"),
      "t._bs_id = s._bs_id and t._id = s._id and t.SourceSystem = s.SourceSystem") \
    .whenNotMatchedInsertAll() \
    .whenMatchedUpdateAll() \
    .execute()

# Flag deletes 

deletes = (quote_line_g.toDF().alias('t')
           .join(quote_line_update
           , ['_bs_id','SourceSystem'] ,how = 'leftsemi')
           .join(quote_line_update ,
            ['_bs_id','_id','SourceSystem'] , how ='leftanti'))

quote_line_g.alias("t").merge(
      deletes.alias("s"),
      "t._bs_id = s._bs_id and t._id = s._id and t.SourceSystem = s.SourceSystem") \
    .whenMatchedUpdate(set = {'isDeleted':lit(1)}) \
    .execute()

#store deleted lines 
deletes.select('_bs_id','_id').coalesce(1).write.format('csv').option("header","true").mode('append').save("/mnt/datalake_raw/batch/sales/bigmachine/deleted_quote_lines")

In [0]:
"""
Quote Line Config & Material Summary
"""

from pyspark.sql.types import StringType , StructField , StructType , TimestampType
from pyspark.sql.functions import split , col , lit , explode, from_csv , first , row_number ,  size , arrays_zip
from pyspark.sql.window import Window
spark.conf.set('spark.sql.caseSensitive', True)
from delta.tables import *  

quoteLineSchema = StructType([StructField("_bs_id",StringType() , False),
                              StructField("_id",StringType() , False),
                               StructField("_sequence_number",StringType() , True),
                              StructField("lineDocNum_line",StringType() , True),
                              StructField("_date_modified",TimestampType(), False) ,
                              StructField("_date_added",TimestampType(), False) ,
                              StructField("_config_attr_info", StructType([StructField("value",StringType(), True)]), True)
                                               
                             ])

emptyDF = (sqlContext.createDataFrame(sc.emptyRDD(), quoteLineSchema).withColumn("SourceSystem",lit("Commerce")).select(
                          "_bs_id"
                          ,"_id"
                          ,"_sequence_number"
                        ,"lineDocNum_line"
                          ,"_date_modified"
   ,"_date_added"
                             ,"_config_attr_info.value"
                             ,"SourceSystem"
                          )
          )
if quote_line_v2_list:
  Quote_Line_v2 = (
                        spark.read
                           .schema(quoteLineSchema)
                          .option("multiline","true")
                          .option("timestampFormat","MM/dd/yyyy HH:mm:SS a")
                          #.json("/mnt/datalake_raw/batch/sales/bigmachine/commerce_v2/quote_line/*/*.json")
                          .json(quote_line_v2_list)
                          .withColumn("SourceSystem" , lit("commerce_v2"))
                          .select(
                          "_bs_id"
                          ,"_id"
                          ,"_sequence_number"
                        ,"lineDocNum_line"
                          ,"_date_modified"
                             ,"_date_added"
                             ,"_config_attr_info.value"
                             ,"SourceSystem"
                          )

                     )

else:
  Quote_Line_v2 = emptyDF

if  quote_line_v1_list:
  Quote_Line_v1 = (
                      spark.read
                         .schema(quoteLineSchema)
                        .option("multiline","true")
                        .option("timestampFormat","MM/dd/yyyy HH:mm:SS a")
                        #.json("/mnt/datalake_raw/batch/sales/bigmachine/commerce_v1/quote_line/*/*.json")
                        .json(quote_line_v1_list)
                         .withColumn("SourceSystem" , lit("commerce_v1"))
                        .select(
                        "_bs_id"
                        ,"_id"
                        ,"_sequence_number"
                      ,"lineDocNum_line"
                        ,"_date_modified"
                           ,"_date_added"
                           ,"_config_attr_info.value"
                          ,"SourceSystem"
                        )

                   )
else:
  Quote_Line_v1 = emptyDF
  
Quote_Line = Quote_Line_v2.unionAll(Quote_Line_v1)

# Drop Duplicates and retieve the most recent lines 
window = Window.partitionBy("SourceSystem","_id").orderBy(Quote_Line["_date_modified"].desc())

Quote_Line = (Quote_Line.dropDuplicates().withColumn("RowNumber" , row_number().over(window))
  .filter("RowNumber == 1")
  .drop("RowNumber"))

quote_line_config = (Quote_Line
.select(split(col("value"), '\\|\\^\\|').alias("ColumnArray"),"_bs_id","_id","_date_modified" ,"SourceSystem" ,"_date_added")
.select("_bs_id","_id","_date_modified","SourceSystem" ,"_date_added",explode("ColumnArray"))
.select("_bs_id","_id","_date_modified" ,"SourceSystem","_date_added",split("col","~")[0].alias("colName") , split("col","~")[2].alias("value"))
.groupBy("_bs_id","_id","_date_modified","SourceSystem" ,"_date_added").pivot("colName").agg(first("value"))
                   
     )
cols =[   
 "_bs_id",
  "_id",
  "_date_modified",
  "_date_added",
   "numberOfCarsInGroupDropdown",
   "existingEquipment",
    "equipmentType",
    "equipmentTypeVIEW",
    "equipmentClass",
    "upspeedOfCar",
    "unitCapacityForRepair",
    "numberOfCarsInGroup",
    "repairNumberOfStops",
    "numberOfFrontOpenings",
    "numberOfRearOpenings",
    "totalOpenings",
    "boardName",
    "controllerBrand",
    "specificControllerModels",
    "controllerType",
    "directReplacement",
    "applicationOfUnit",
    "boardType",
    "purification",
    "bACNet_material",
    "cabHeightSelection",
    "compensationType",
    "controllerLocation",
    "counterweightSafety",
    "flooringThickness",
    "pitDepthInFeet",
    "interimMaintenanceTotalPrice",
    "interimMaintenanceMonths",
    "freeServiceTotalPrice",
    "freeServiceMonths",
    "appliedSRTDiscounts",
    "mAXFactoryLeadTime",
    "batch1LeadTime",
    "batch2LeadTime",
    "batch3LeadTime",
    "equipmentClassification",
    "rearDoorTypeAndHand",
    "driveType",
    "machineRoomLocation",
    "jackType",
    "capacity",
    "frontDoorTypeAndHand",
    "frontOpenings",
    "unitNumberOfStops",
    "rearOpenings",
    "futureSpeed",
    "travelInFeet",
    "totalFactoryMaterialCost",
    "otherMaterialCosts",
    "miscellaneousLaborHours",
    "adjustingInspectionNIMHours",
    "jHAHours",
    "totalCabHours",
    "totalCarDoorEquipmentHours",
    "totalCarFixturesHours",
    "totalCarHours",
    "totalControllerTaskHours",
    "totalGovernorHours",
    "totalHallFixturesHours",
    "totalHoistwayDoorEquipmentHours",
    "totalHoistwayEquipmentHours",
    "totalJackHours",
    "totalMachineHours",
    "totalPitEquipmentHours",
    "totalPowerUnitHours",
    "totalStandardJobTasksHours",
    "totalWorkByOthersHours",
    "aGILEDestinationControls",
    "roping",
    "selectedRoping",
    "ropeSize",
    "finalRopeSize",
    "finalRopeQuantity",
    "existingCarWeight",
    "existingMotorHP",
    "existingMotorRPM",
    "mainlineVoltage",
    "overheadInFeet",
    "futureOverheadInFeet",
    "orderType",
    "existingControllerManufacturer",
    "existingControllerModel",
    "existingControllerType", 
    "nickname_ServiceUnitsSet",
    "legalID_ServiceUnitsSet",
    "numberOfStops_ServiceUnitsSet",
    "frontOpenings_ServiceUnitsSet",
    "customerNumber_ServiceUnitsSet",
    "branch_ServiceUnitsSet",
    "routenumber_ServiceUnitsSet",
    "siteAddressLine1_ServiceUnitsSet",
    "siteAddressLine2_ServiceUnitsSet",
    "city_ServiceUnitsSet",
    "state_ServiceUnitsSet",
    "zipCode_ServiceUnitsSet",
    "rearOpenings_ServiceUnitsSet",
    "productType_ServiceUnitsSet",
    "applicationOfUnit_ServiceUnitsSet",
    "manufacturer_ServiceUnitsSet",
    "oEMSerialNumber_ServiceUnitsSet",
    "controllerManufacturer_ServiceUnitsSet",
    "controllerModel__ServiceUnitsSet",
    "controllerType_ServiceUnitsSet",
    "criticalUnit_ServiceUnitsSet",
    "pumpMotorOEM_ServiceUnitsSet",
    "driveConfiguration_ServiceUnitsSet",
    "speed_ServiceUnitsSet",
    "periodicTestCovered_ServiceUnitsSet",
    "annualTestCovered_ServiceUnitsSet",
    "lastAnnualInspection_ServiceUnitsSet",
    "lastPeriodicInspectionDate_ServiceUnitsSet",
    "nextSafetyInspectionDate_ServiceUnitsSet",
    "lastCategory5InspectionDate_ServiceUnitsSet",
    "capacity_ServiceUnitsSet",
    "machineRoomLocation_ServiceUnitsSet",
    "pumpMotorType_ServiceUnitsSet",
    "jackType_ServiceUnitsSet",
    "accountName_ServiceUnitsSet",
    "doorComplexity_ServiceUnitsSet",
    "equipmentType_ServiceUnitsSet",
    "motorStarter_ServiceUnitsSet",
    "equipmentCondition_ServiceUnitsSet",
    "oracleSerialNumber_ServiceUnitsSet",
    "equipmentTypeVIEW_ServiceUnitsSet",
    "equipmentClass_ServiceUnitsSet",
    "equipmentManufacturer_ServiceUnitsSet",
    "equipmentModel_ServiceUnitsSet",
    "machineManufacturer_ServiceUnitsSet",
    "customerNumber_UnitsSet",
    "unitNickName_UnitsSet",
    "unitNickName_UnitsSet_baseTab",
    "factoryJob_UnitsSet",
    "equipmentTypeVIEW_UnitsSet",
    "siteAddressLine1_UnitsSet",
    "siteAddressLine2_UnitsSet",
    "oEMSerialNumber_UnitsSet",
    "city_UnitsSet",
    "zipCode",
    "legalID_UnitsSet",
    "oracleSerialNumber_UnitsSet",
    "state_UnitsSet",
    "buildingTypeName_UnitsSet",
    "frontOpenings_UnitsSet",
    "rearOpenings_UnitsSet",
    "unitNicknameReadonly_UnitsSet",
    "accountName_UnitsSet",
    "selectUnit_UnitsSet"
  ]
for column in cols:
  if column not in quote_line_config.columns:
    quote_line_config= quote_line_config.withColumn(column,lit(""))
  
quote_line_config_details_update = quote_line_config.select(*cols).drop_duplicates()


quote_line_material_summary_update = (quote_line_config.withColumn("zipped",arrays_zip(
                                           split(col("proposalPartDescription_materialSummary"),'\$,\$').alias("Desc") #0
                                           ,split(col("baseMaterialCost_materialSummary"),'\$,\$').alias("BaseMaterialCost") #1
                                           ,split(col("qty_materialSummary"),'\$,\$').alias("qty")  #2
                                           ,split(col("groupMaterial_materialSummary"),'\$,\$').alias("groupMaterial")  #3
                                           ,split(col("includeInTheProposal_materialSummary"),'\$,\$').alias("includeInTheProposal")  #4
                                           ,split(col("level1_materialSummary"),'\$,\$').alias("level1")  #5
                                           ,split(col("level2_materialSummary"),'\$,\$').alias("level2")  #6
                                           ,split(col("level3_materialSummary"),'\$,\$').alias("level3")  #7
                                           ,split(col("partNumber_materialSummary"),'\$,\$').alias("partNumber")  #8
                                           ,split(col("section_materialSummary"),'\$,\$').alias("section")  #9
                                           ,split(col("selectedPartDescription_materialSummary"),'\$,\$').alias("selectedPartDescription")  #10
                                           ,split(col("sequence_materialSummary"),'\$,\$').alias("sequence")  #11
                                           ,split(col("surveyKeys_materialSummary"),'\$,\$').alias("surveyKeys")  #12
                                           ,split(col("tKECost_materialSummary"),'\$,\$').alias("tKECost")  #13
                                           ,split(col("totalMaterialCost_materialSummary"),'\$,\$').alias("totalMaterialCost")  #14
                                           ,split(col("vendor_materialSummary"),'\$,\$').alias("vendor")  #15
                                          ))
        .withColumn("zipped" ,explode(col("zipped"))) 
             
        .select(
            "_bs_id"
           ,"_id"
           ,"_date_modified"
           ,"_date_added"
                ,col("zipped.Desc").alias("proposalPartDescription") 
                ,col("zipped.BaseMaterialCost").alias("baseMaterialCost") 
                ,col("zipped.qty").alias("qty") 
                ,col("zipped.groupMaterial").alias("groupMaterial") 
                ,col("zipped.includeInTheProposal").alias("includeInTheProposal") 
                ,col("zipped.level1").alias("level1") 
                ,col("zipped.level2").alias("level2")
                ,col("zipped.level3").alias("level3")
                ,col("zipped.partNumber").alias("partNumber") 
                ,col("zipped.section").alias("section")
                ,col("zipped.selectedPartDescription").alias("selectedPartDescription")
                ,col("zipped.sequence").alias("sequence") 
                ,col("zipped.surveyKeys").alias("surveyKeys")
                ,col("zipped.tKECost").alias("tKECost")
                ,col("zipped.totalMaterialCost").alias("totalMaterialCost") 
                ,col("zipped.vendor").alias("vendor")
                ).dropDuplicates())





# Merge quote line config & material Summary : Update changed quotes and insert newly created accounts 

quote_line_config_details_g = DeltaTable.forPath(spark, "/mnt/datalake_premium/cpq/quote_line_config") 
quote_line_material_summary_g = DeltaTable.forPath(spark, "/mnt/datalake_premium/cpq/quote_line_material_summary") 

quote_line_config_details_g.alias("t").merge(
quote_line_config_details_update.alias("s"),
"t._bs_id = s._bs_id  and t._id = s._id") \
.whenMatchedDelete() \
.execute()

quote_line_config_details_update.write.format('delta').mode('append').save('/mnt/datalake_premium/cpq/quote_line_config')

quote_line_material_summary_update.createOrReplaceTempView('quote_line_material_summary_update')
spark.sql("""
DELETE FROM ods.cpq_quote_line_material_summary WHERE _bs_id in (select distinct _bs_id from quote_line_material_summary_update)
""")
# quote_line_material_summary_g.alias("t").merge(
# quote_line_material_summary_update.alias("s"),
# "t._bs_id = s._bs_id  and t._id = s._id") \
# .whenMatchedDelete() \
# .execute()

quote_line_material_summary_update.write.format('delta').mode('append').save('/mnt/datalake_premium/cpq/quote_line_material_summary')


In [0]:
dbutils.notebook.exit("Job Completed Successfuly!")

Job Completed Successfuly!

In [0]:
%sql
create or replace view ods.v_cpq_highcabSubsidy as
Select Distinct 
  h.crmOpportunityId_quote
, h.quoteNumber_quote
, (Case When h.totalHighCabSubsidy_quote is null then 0 else h.totalHighCabSubsidy_quote end) totalHighCabSubsidy

from ods.cpq_quote_header h
Where 1=1
and (Case When h.totalHighCabSubsidy_quote is null then 0 else h.totalHighCabSubsidy_quote end) <>0